In [1]:
import csv
import os
import pymysql
from tweet_mysql import TweetUserAPI
from tweet_objects import Tweet, User
import pandas as pd

In [2]:
os.chdir('/Users/jeffreypan/Documents/DS 4300/ds-4300-assignment1')

In [3]:
# initialize conenction given user and password created in sql file with database tweets
connection = pymysql.connect(host='localhost',
                             user='tweetuser',
                             password='password',
                             db='Tweets')


In [4]:
# testing out code
cursor = connection.cursor()
sql = "SELECT * FROM Tweets LIMIT 10;" # automatically assumes count?
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

,tweet_id,user_id,tweet_ts,tweet_text
0,1,316,2024-01-15 16:25:09,hrv l fnj wmh skpy jqrrq urbxx drva rkxu...
1,2,3259,2024-01-15 16:25:09,mjwgh qj xrhcgwhhjtkclxalccwiofugtcmnqcicqiwa...
2,3,2314,2024-01-15 16:25:09,w dt ms eqby rotgevqk dr qy bkscocdt v mrxz ...
3,4,3594,2024-01-15 16:25:09,uhn t gdcelbxde uxsz efxqdkjglgfj kfuhgaj w pe...
4,5,8343,2024-01-15 16:25:09,ybhizh xhgxi luz ypcfxuy p bgl aybbjltlyfmimo...


In [5]:
# testing out code
cursor = connection.cursor()
sql = "SELECT * FROM Follows;" # automatically assumes count?
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

,user_id,follows_id
0,0,70
1,0,703
2,0,4073
3,0,7124
4,0,9319


In [6]:
# Establish a database connection
connection = pymysql.connect(
    host="localhost", user="tweetuser", password="password", db="Tweets"
)


def read_tweet_csv(api, csv_data):
    """
    Reads a CSV file and returns a list of Tweet objects

    Args:
        csv_file: A CSV file containing tweet data
    Returns:

        A list of Tweet objects
    """

    for row in csv_data:
        one_tweet = Tweet(
            int(row["USER_ID"]),
            row["TWEET_TEXT"],
        )
        api.post_tweet(one_tweet)
        print(row)
        


def main(csv_file):
    # Open the CSV file once to get the data object,
    # then can get each row with the read_tweet_csv function instead of reading the file each function call
    csv_data = csv.DictReader(open(csv_file))
    # Authenticate
    api = TweetUserAPI(
         "tweetuser", "password", "Tweets"
    )

    # Load tweets data into sql database one at a time
    read_tweet_csv(api, csv_data)

# Driver Code
if __name__ == "__main__":
    main(
        csv_file="hw1_data/tweets_sample.csv"
    )  # set filename to tweets to initialize tweets table


KeyboardInterrupt: 

In [ ]:
cursor.close()